## Install dependencies

In [1]:
!pip install --upgrade pip
!pip install transformers torch accelerate bitsandbytes
!pip install --upgrade transformers

!pip install huggingface-hub
!pip install python-dotenv


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Authorization

In [2]:
from huggingface_hub import login
import os
from dotenv import load_dotenv

# Load environment variables from config.env
load_dotenv("config.env")

os.environ["HF_HOME"] = "/projects/ec12/ec-sindrre/cache" 

# Put into env.config file
access_token_read = os.environ.get('HF_API_KEY')
login(token = access_token_read)

if access_token_read:
    login(token=access_token_read)
    print("Logged in successfully!")
else:
    print("HF_API_KEY is not set in your environment variables.")


Logged in successfully!


## Prepare data

In [12]:
import json
from Scripts.few_shot.split_dataset_on_shot import 
# Load datasets
def load_dataset(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)
        
num_shot = 10 # currently either 10 or 5 datasets

train_data = load_dataset('./Data/Few-shot/train_' + str(num_shot) + '_shot.json')
val_data = load_dataset('./Data/Few-shot/validation_' + str(num_shot) + '_shot.json')
test_data = load_dataset('./Data/Few-shot/test_' + str(num_shot) + '_shot.json')

# Extract prompts and library_functions (responses) from the dataset
def extract_prompts_and_responses(data):
    # Prompts from the first element of 'prompts', with additional instruction
    prompts = [f"Identify the library functions to be used in this example: {item['prompts'][0]}\n {item['prompts'][1]}" for item in data]
    # Responses from the library_functions list
    responses = [", ".join(item["library_functions"]) for item in data]
    return prompts, responses

def used_libraries_to_string(data):
    name_doc_string = ""
    for func in data:
        name_doc_string += f"Name: {func['function_name']}\nDocumentation: {func['doc']}\n\n"
    return name_doc_string
    
# Extract training, validation, and test data
train_prompts, train_responses = extract_prompts_and_responses(train_data)  # Use as examples for few-shot learning
val_prompts, val_responses = extract_prompts_and_responses(val_data)  # Validation set
test_prompts, test_responses = extract_prompts_and_responses(test_data)  # Test set

# Extract all unique nodes (library_functions) across datasets
used_libraries_json = load_dataset("./Data/Few-shot/libraries_" + str(num_shot) + "_shot.json")
explained_used_libraries = used_libraries_to_string(used_libraries_json)

print(f"train set samples: {len(train_prompts)}")
print(f"Validation set samples: {len(val_prompts)}")
print(f"test set samples: {len(test_prompts)}")


ModuleNotFoundError: No module named 'install_dependencies'

## Create Prompt

In [4]:
# Function to create few-shot prompt
def create_few_shot_prompt(train_prompts, train_responses, input_prompt):
    Context = "You are going to solve some programming tasks for node-based programming language. Use minimal amount of library functions to solve the tasks.\n" 
    node_list = f"Only use the following library functions:\n {explained_used_libraries}\n\n"
    formatted_prompt = node_list
    for i, (prompt, response) in enumerate(zip(train_prompts, train_responses)):
        formatted_prompt += f"Example {i+1}:\nPrompt: {prompt}\nResponse: {response}\n\n"
    formatted_prompt += f"Task:\n{input_prompt}\nResponse:"
    return formatted_prompt


## Test some Models, with different seeds, temperatures, top_ps

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, set_seed
import torch

print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should return the number of GPUs
print(torch.cuda.current_device())  # Should return the current GPU index (e.g., 0)
print(torch.cuda.get_device_name(0))  # Should return the name of your GPU

if torch.cuda.is_available():
    torch_dtype = torch.float16
else:
    torch_dtype = torch.float32  # Use FP32 on CPU

# Function to generate and evaluate responses
def evaluate_with_pipeline(pipeline, prompts, responses, max_new_tokens=50, temperature=0.7, top_p=0.9):
    correct = 0
    total = len(prompts)

    for index, (prompt, true_response) in enumerate(zip(prompts, responses)):
        full_prompt = create_few_shot_prompt(train_prompts, train_responses, prompt)
        generated = pipeline(
            full_prompt,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            return_full_text=False,
            num_return_sequences=1,  # Ensure only one response
        )[0]["generated_text"]
        
        print(f"\n\nSample: {index}")
        print(f"Prompt: {prompt}")
        print(f"Generated response: {generated}")
        print(f"True response: {true_response}")

         # Tokenize both responses into words
        generated_words = set(generated.replace(",", "").split())
        true_response_words = set(true_response.replace(",", "").split())
        library_functions = set(item['function_name'] for item in used_libraries_json)
        print(f"Found these valid functions in output: {generated_words.intersection(library_functions)}")
        print(f"Correct nodes is: {true_response_words}")
        # Check if there are any common words
        if generated_words.intersection(library_functions) == true_response_words:
            print("correct response")
            correct += 1
        else:
            print("Invalid response")
        
    return correct / total

# List of models to test
models_to_test = [
    {"name": "meta-llama/Meta-Llama-3.1-8B", "cache_dir": "/projects/ec12/ec-sindrre/cache"},
    # {"name": "mistralai/Mistral-Small-Instruct-2409", "cache_dir": "/projects/ec12/ec-sindrre/cache"},
    # {"name": "meta-llama/Llama-3.3-70B-Instruct", "cache_dir": "/projects/ec12/ec-sindrre/cache"},
    # {"name": "meta-llama/CodeLlama-70b-Instruct-hf", "cache_dir": "/projects/ec12/ec-sindrre/cache"},
    # {"name": "meta-llama/Llama-3.2-90B-Vision-Instruct", "cache_dir": "/projects/ec12/ec-sindrre/cache"}
]
results = {}
seeds = [3, 75, 346]  # List of some random chosen seeds for consistent variability testing

for model_info in models_to_test:
    print(f"Testing model: {model_info['name']}...")

    # Load model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(
        model_info["name"],
        cache_dir=model_info["cache_dir"],
        torch_dtype=torch_dtype,
        device_map="auto"
    )

    tokenizer = AutoTokenizer.from_pretrained(
        model_info["name"],
        cache_dir=model_info["cache_dir"]
    )
    tokenizer.pad_token_id = tokenizer.eos_token_id

    # Create pipeline for the model
    text_gen_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch_dtype,
        device_map="auto"
    )
    # Test with different seeds
    for seed in seeds:
        print(f"\nTesting with Seed: {seed}")
        set_seed(seed)
        
        # Validation phase
        temperatures = [0.5, 0.7, 0.9]
        top_ps = [0.2, 0.5, 1.0]
        best_accuracy = 0
        best_params = {"temperature": 0.7, "top_p": 0.9}  # Default values
        
        for temp in temperatures:
            for top_p in top_ps:
                accuracy = evaluate_with_pipeline(text_gen_pipeline, val_prompts, val_responses, temperature=temp, top_p=top_p)
                print(f"Tested with temp={temp} and top_p={top_p}. Gave accuracy={accuracy}")
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {"temperature": temp, "top_p": top_p}
    
        print(f"Best Hyperparameters for {model_info['name']}: {best_params}, Validation Accuracy: {best_accuracy:.2f}")
    
        # Test phase
        if not best_params:
            print("No valid hyperparameters found during validation; using default values.")
            best_params = {"temperature": 0.7, "top_p": 0.9}
        
        test_accuracy = evaluate_with_pipeline(
            text_gen_pipeline,
            test_prompts,
            test_responses,
            temperature=best_params["temperature"],
            top_p=best_params["top_p"]
        )
    
        print(f"Test Accuracy for {model_info['name']}: {test_accuracy:.2f}")
    
        # Store results
        if model_info["name"] not in results:
            results[model_info["name"]] = []
            results[model_info["name"]].append({
                "seed": seed,
                "validation_accuracy": best_accuracy,
                "test_accuracy": test_accuracy
            })

# Print summary of results
print("\nFinal Results:")
for model_name, metrics in results.items():
    print(f"{model_name}:")
    for run in metrics:
        print(f"  Seed {run['seed']}: Validation Accuracy: {run['validation_accuracy']:.2f}, Test Accuracy: {run['test_accuracy']:.2f}")

True
1
0
NVIDIA H100 PCIe MIG 1g.20gb
Testing model: meta-llama/Meta-Llama-3.1-8B...


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x15039d681c50>>
Traceback (most recent call last):
  File "/cluster/software/EL9/easybuild/software/jupyter-server/2.14.0-GCCcore-13.2.0/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0



Testing with Seed: 3


Sample: 0
Prompt: Identify the library functions to be used in this example: Write a python function to check whether a list of numbers contains only one distinct element or not.
 
Generated response:  Iteration.Reduce, Iteration.Filter, Logic.Equal, Logic.NotEqual, List.Contains, Logic.Not, Math.Modulo, Math.Add, Math.Sub, Math.Mul, Math.Div, Math.Expression, List.GetAt, List.IsEmpty, List
True response: Sets.Intersection, List.Length, Logic.Equal
Found these valid functions in output: {'Logic.Not', 'Math.Modulo', 'List.IsEmpty', 'Math.Expression', 'List.Contains', 'Iteration.Reduce', 'Math.Sub', 'Logic.NotEqual', 'Math.Add', 'Logic.Equal', 'List.GetAt', 'Math.Mul', 'Iteration.Filter'}
Correct nodes is: {'Logic.Equal', 'List.Length', 'Sets.Intersection'}
Invalid response


Sample: 1
Prompt: Identify the library functions to be used in this example: Write a function to find the index of the first occurrence of a given number in a sorted array.
 
Generated respon

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset




Sample: 0
Prompt: Identify the library functions to be used in this example: Write a python function to check whether a list of numbers contains only one distinct element or not.
 
Generated response:  Iteration.Reduce, Iteration.Filter, Logic.Equal, Logic.NotEqual, List.Contains, Logic.Not, Math.Modulo, Math.Add, Math.Sub, Math.Mul, Math.Div, Math.Expression, List.GetAt, List.IsEmpty, List
True response: Sets.Intersection, List.Length, Logic.Equal
Found these valid functions in output: {'Logic.Not', 'Math.Modulo', 'List.IsEmpty', 'Math.Expression', 'List.Contains', 'Iteration.Reduce', 'Math.Sub', 'Logic.NotEqual', 'Math.Add', 'Logic.Equal', 'List.GetAt', 'Math.Mul', 'Iteration.Filter'}
Correct nodes is: {'Logic.Equal', 'List.Length', 'Sets.Intersection'}
Invalid response


Sample: 1
Prompt: Identify the library functions to be used in this example: Write a function to find the index of the first occurrence of a given number in a sorted array.
 
Generated response:  List.GenerateRang